In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import regex as re
import numpy as np
import requests
import string
import time 

In [2]:
domain = 'http://amazon.com'
URL = 'https://www.amazon.com/s?k=adidas+shorts+for+men&ref=nb_sb_noss_1'
driver = webdriver.Chrome(executable_path ='/Applications/chromedriver')
driver.get(URL)
subhtml = driver.page_source
soup = BeautifulSoup(subhtml, "html.parser")
df = pd.DataFrame()
mylist=[]

while True:
    
    for i in soup.find_all(class_='a-section a-spacing-medium'):
        
        #go inside product page
     
        nextlink = domain + i.find(class_='a-link-normal a-text-normal')['href']
        print(nextlink)
        driver.get(nextlink)
        time.sleep(5)
        temp = driver.page_source
        soup = BeautifulSoup(temp, "html.parser")
        
        #0. extract image link
        try:
            #images = soup.findAll('img')
            images = soup.findAll('div',{'class':'imgTagWrapper'})[0].findAll('img')[0].get('src')
            #for image in images:
            print(images['src'])
            mylist.append(images['src'])
            #image = soup.find(id='imgTagWrapperId').data-a-dynamic-image
            #print(image)            
        except:
            product_image = np.nan
            print('NO product_images')

        #1. extract brand name
        try:
            brand = soup.find_all(id='bylineInfo')[0].text
            if 'Visit' in brand:
                brand = re.match('(?:.{0,10})(.+)(?:\s.+)', brand).group(1)
                print(brand)
            elif 'Brand:' in brand:
                brand = re.match('^Brand\:\s(.+)', brand).group()
            else:
                print('NO brand name')
        except:
            brand = np.nan
            print('NO Brand')
            
        
        #2. extract product name        
        try:   
            product_name = soup.find(id='productTitle').text.replace('\n','').replace(brand+' ','')
            print(product_name)
        except:
            product_name = np.nan
            print('NO product_name')
            
        #3. extract category
        try:
            cate = soup.find(id='wayfinding-breadcrumbs_feature_div').find_all(class_='a-list-item')[6].text.replace('\n','')
            print('Categorie: '+cate)
        except:
            cate = np.nan
            print('NO CATE') 

            
        #4. extract original price /rmb covert to hkd!
        try:
            original_price = soup.find(class_='priceBlockStrikePriceString a-text-strike').text
            print('original price(USD): '+original_price)
            
        except:
            
            try:
                original_price = soup.find(class_='a-size-medium a-color-price priceBlockBuyingPriceString').text
                print('original price(USD): '+original_price)
            except:
                original_price = np.nan
                print('NO original_price')
                
               
                
        #5. extract discounted price /rmb covert to hkd!
        try:
            discounted_price = soup.find(class_='a-size-medium a-color-price priceBlockBuyingPriceString').text
            print('discounted_price(USD): '+discounted_price)
            
        except:
            
            try:
                discounted_price = original_price
                print('discounted_price(USD): '+discounted_price)
            except:
                discounted_price = np.nan
                print('NO discounted_price')
                
        #6. extract promotiona message
#        try:
#            promotion_message = soup.find_all(id='promoPriceBlockMessage_feature_div')[1].text.replace('\n','').replace('  Shop items','')
#            print('Promotion Message: '+promotion_message)
#        except:
#            try:
#                promotion_message = soup.find(id='vpcButton').text.replace('\n','')
#                print('Promotion Message: '+promotion_message)
#            except:
#                promotion_message = np.nan
#                print('NO Promo')
 
        #7. extract no of reviews(ratings)
#        try:
#            no_of_reviews = soup.find(id='acrCustomerReviewText').text.replace(',',"")
#            no_of_reviews = re.match('(\d+)', no_of_reviews).group()
#            print('no_of_reviews: '+no_of_reviews)
#        except:
#            no_of_reviews = np.nan
#            print('NO no_of_reviews')


        #8. extract ratings    
        try:
            ratings = soup.find(class_='a-icon-alt').text
            ratings = re.match('(\d\.?\d?)(?:\s.+)', ratings).group(1)
            print('ratings: ' + ratings)
        except:
            ratings = np.nan
            print('NO ratings')
            
        #9. product url
        try:
            product_url = domain + i.find('a')['href']
            # print(product_url)
        except:
            product_url = np.nan
        
        #column_values = pd.Series(mylist)
        #df = df.insert(loc=0, column='Images', value=column_values)
        
        df = df.append({'Images':images,
                        'Brand':brand,
                        'Product Name':product_name,
                        'Category':cate,
                        'Original Price':original_price,
                        'Discounted Price':discounted_price,
                        'Ratings':ratings,
                        'Product URL':product_url}, ignore_index = True)
#                        'No. of Reviews':no_of_reviews,
#                        'Promotion Message':promotion_message
        
        
    #going back to main page after looping    
    temp = driver.get(URL) 
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, "html.parser")

    #going to the next page
    temp2 = soup.find(class_='a-last').a['href']
    URL = domain + temp2
    driver.get(URL)
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, "html.parser")
    
    

http://amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A00811793TL70GIJBEZEE&url=%2Fadidas-Designed-3-Stripes-Shorts-XX-Large%2Fdp%2FB07DXK9RSD%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dadidas%2Bshorts%2Bfor%2Bmen%26qid%3D1607043498%26sr%3D8-1-spons%26psc%3D1&qualifier=1607043498&id=5617008412461679&widgetName=sp_atf
NO product_images
adidas
Men's Designed 2 Move 3-Stripes Cool Shorts
Categorie:                 Clothing            
original price(USD):  $30.00
discounted_price(USD): $22.50
ratings: 4.6
http://amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0370286213IIYTKTTB20&url=%2Fadidas-Sport-3-Stripes-Short-X-Small%2Fdp%2FB07M9637F2%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dadidas%2Bshorts%2Bfor%2Bmen%26qid%3D1607043498%26sr%3D8-2-spons%26psc%3D1&qualifier=1607043498&id=5617008412461679&widgetName=sp_atf
NO product_images
adidas
Men's Sport 3-Stripes Shorts
Categorie:                 Basketball   

NO product_images
adidas
Men's Condivo 20 Shorts
Categorie:                 Soccer            
original price(USD): $15.23 - $41.95
discounted_price(USD): $15.23 - $41.95
ratings: 4.4
http://amazon.com/adidas-Soccer-Squadra-Shorts-Medium/dp/B01H7VR69C/ref=sr_1_23?dchild=1&keywords=adidas+shorts+for+men&qid=1607043498&sr=8-23
NO product_images
adidas
Men's Squadra 17 Shorts
Categorie:                 Soccer            
original price(USD): $9.99 - $49.34
discounted_price(USD): $9.99 - $49.34
ratings: 4.5
http://amazon.com/adidas-Essentials-3-Stripes-Fleece-Short/dp/B07F5VZXVS/ref=sr_1_24?dchild=1&keywords=adidas+shorts+for+men&qid=1607043498&sr=8-24
NO product_images
adidas
Men's Essentials 3-Stripes Fleece Shorts
Categorie:                 Active            
original price(USD): $23.32 - $61.45
discounted_price(USD): $23.32 - $61.45
ratings: 4.7
http://amazon.com/adidas-Creator-Short-Scarlet-Small/dp/B07M8P3466/ref=sr_1_25?dchild=1&keywords=adidas+shorts+for+men&qid=1607043498&sr=8-25


NO product_images
adidas
Men's Axis Woven Training Shorts
Categorie:                 Running            
original price(USD): $23.37 - $52.65
discounted_price(USD): $23.37 - $52.65
ratings: 4.0
http://amazon.com/adidas-Sport-11-Inch-Woven-Short/dp/B07Y46BQG4/ref=sr_1_46?dchild=1&keywords=adidas+shorts+for+men&qid=1607043498&sr=8-46
NO product_images
adidas
Sport ID 11-Inch Woven Short - Men's Multi-Sport
Categorie:                 Active            
original price(USD): $19.00 - $39.00
discounted_price(USD): $19.00 - $39.00
NO ratings
http://amazon.com/adidas-Originals-Trefoil-Essentials-Shorts/dp/B087QDPLQM/ref=sr_1_47?dchild=1&keywords=adidas+shorts+for+men&qid=1607043498&sr=8-47
NO product_images
adidas Originals
Men's Trefoil Essentials Shorts
Categorie:                 Active            
original price(USD): $28.23 - $65.88
discounted_price(USD): $28.23 - $65.88
ratings: 5.0
http://amazon.com/adidas-Soccer-Alphaskin-Shorts-Tights/dp/B078L7NLVW/ref=sr_1_48?dchild=1&keywords=adidas+

NO product_images
MO GOOD Mens Casual Shorts Workout Fashion Comfy Shorts Breathable Big and Tall Shorts
Categorie:                 Men            
original price(USD): $19.95
discounted_price(USD): $19.95
ratings: 4.4
http://amazon.com/adidas-Performance-Practice-Shorts-X-Large/dp/B00R5511GK/ref=sr_1_53?dchild=1&keywords=adidas+shorts+for+men&qid=1607043963&sr=8-53
NO product_images
adidas
Men's Team Speed Practice Shorts
Categorie:                 Basketball            
NO original_price
NO discounted_price
ratings: 4.6
http://amazon.com/adidas-Sport-Basketball-Collegiate-XX-Large/dp/B07LDBLPPT/ref=sr_1_54?dchild=1&keywords=adidas+shorts+for+men&qid=1607043963&sr=8-54
NO product_images
adidas
Men's Badge of Sport Shorts
Categorie:                 Basketball            
original price(USD): $22.99 - $30.00
discounted_price(USD): $22.99 - $30.00
ratings: 4.8
http://amazon.com/DEVOPS-10-inch-Loose-Fit-Workout-Training/dp/B07SPDRP9L/ref=sr_1_55?dchild=1&keywords=adidas+shorts+for+men&qid

NO product_images
adidas
Men's Axis Knit Training Shorts
Categorie:                 Active            
original price(USD): $24.56 - $44.88
discounted_price(USD): $24.56 - $44.88
ratings: 5.0
http://amazon.com/adidas-Essentials-3-Stripe-Shorts-Black/dp/B00IY9D42A/ref=sr_1_76?dchild=1&keywords=adidas+shorts+for+men&qid=1607043963&sr=8-76
NO product_images
adidas
Men's Essentials 3-Stripe Shorts
Categorie:                 Running            
original price(USD): $11.78 - $52.01
discounted_price(USD): $11.78 - $52.01
ratings: 4.3
http://amazon.com/adidas-Training-Ultimate-Transitional-Collegiate/dp/B077ZCS7CX/ref=sr_1_77?dchild=1&keywords=adidas+shorts+for+men&qid=1607043963&sr=8-77
NO product_images
adidas
Men's Training Ultimate Transitional Short
Categorie:                 Clothing            
original price(USD): $19.68 - $50.00
discounted_price(USD): $19.68 - $50.00
ratings: 4.3
http://amazon.com/adidas-Short-Primeblue-White-Black/dp/B07S3MRTYV/ref=sr_1_78?dchild=1&keywords=adidas+sh

...

NO product_images
adidas
Women's Tastigo 19 Shorts
Categorie:                 Soccer            
original price(USD): $18.29
discounted_price(USD): $18.29
ratings: 4.6
http://amazon.com/adidas-Performance-Energy-Inseam-Shorts/dp/B074584GRH/ref=sr_1_264?dchild=1&keywords=adidas+shorts+for+men&qid=1607045808&sr=8-264
NO product_images
adidas
Men's Running Ultra Energy Shorts
Categorie:                 Running            
original price(USD): $54.88
discounted_price(USD): $54.88
NO ratings
http://amazon.com/adidas-Mens-Khaki-Cargo-Short/dp/B07NPTYSH1/ref=sr_1_265?dchild=1&keywords=adidas+shorts+for+men&qid=1607045808&sr=8-265
NO product_images
adidas
Men's Cargo Shorts, Color Variation
Categorie:                 Active            
NO original_price
NO discounted_price
ratings: 4.1
http://amazon.com/adidas-Originals-Outline-Shorts-Black/dp/B07S64RHRM/ref=sr_1_266?dchild=1&keywords=adidas+shorts+for+men&qid=1607045808&sr=8-266
NO product_images
adidas Originals
Men's Outline Shorts
Categori

NO product_images
adidas
Men's 4krft Tech 3-Stripes Woven Short
Categorie:                 Clothing            
original price(USD): $26.00 - $58.10
discounted_price(USD): $26.00 - $58.10
ratings: 4.3
http://amazon.com/adidas-Shorts-Black-Olive-Medium/dp/B07KVJZHFH/ref=sr_1_287?dchild=1&keywords=adidas+shorts+for+men&qid=1607045808&sr=8-287
NO product_images
adidas
Men's Own The Run Shorts, Black/Olive, Medium
Categorie:                 Running            
original price(USD):  $35.00
discounted_price(USD): $26.81
NO ratings
http://amazon.com/adidas-Mens-Striker-Shorts-White/dp/B00EDQGQTG/ref=sr_1_288?dchild=1&keywords=adidas+shorts+for+men&qid=1607045808&sr=8-288
NO product_images
adidas
Performance Men's Striker Athletic Short
Categorie:                 Soccer            
original price(USD): $14.99 - $46.00
discounted_price(USD): $14.99 - $46.00
ratings: 4.1
http://amazon.com/adidas-Originals-Spirit-Shorts-Collegiate/dp/B07F3BHWWK/ref=sr_1_289?dchild=1&keywords=adidas+shorts+for+men

NO product_images
adidas
Unisex-Child Youth Parma 16 Shorts
Categorie:                 Soccer            
original price(USD): $10.26 - $39.99
discounted_price(USD): $10.26 - $39.99
ratings: 3.6
http://amazon.com/adidas-Running-Response-2X-Large-Bright/dp/B07BBS5QY3/ref=sr_1_295?dchild=1&keywords=adidas+shorts+for+men&qid=1607046266&sr=8-295
NO product_images
adidas
Men's Running Response Short
Categorie:                 Clothing            
original price(USD): $35.00
discounted_price(USD): $35.00
ratings: 4.4
http://amazon.com/adidas-Training-Speedbreaker-Jacquard-Medium/dp/B077ZCS7BV/ref=sr_1_296?dchild=1&keywords=adidas+shorts+for+men&qid=1607046266&sr=8-296
NO product_images
adidas
Men's Training Speedbreaker Tech Moto Jacquard Short
Categorie:                 Clothing            
original price(USD): $35.00
discounted_price(USD): $35.00
ratings: 5.0
http://amazon.com/adidas-Ultimate-3-Stripes-Short-Khaki/dp/B01L1URPJQ/ref=sr_1_297?dchild=1&keywords=adidas+shorts+for+men&qid=16070

NO product_images
Neleus
Men's Lightweight Workout Running Athletic Shorts with Pockets
Categorie:                 Running            
original price(USD): $37.89
discounted_price(USD): $37.89
ratings: 4.4
http://amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_mtf_aps_sr_pg7_1?ie=UTF8&adId=A01485922HY3W9BBW1QX5&url=%2FBROKIG-Bodybuilding-Running-Lightweight-Waistband%2Fdp%2FB07L2V9KM4%2Fref%3Dsr_1_316_sspa%3Fdchild%3D1%26keywords%3Dadidas%2Bshorts%2Bfor%2Bmen%26qid%3D1607046266%26sr%3D8-316-spons%26psc%3D1&qualifier=1607046270&id=291255934726260&widgetName=sp_mtf
NO product_images
BROKIG
Men's 5" Gym Bodybuilding Shorts Running Workout Lightweight Shorts Elastic Waistband with Pockets
Categorie:                 Running            
original price(USD): $19.99
discounted_price(USD): $19.99
ratings: 4.4
http://amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg7_1?ie=UTF8&adId=A0851905DR51AYGRUCO5&url=%2FDry-Fit-Gym-Shorts-Men%2Fdp%2FB07KXCJMY5%2Fref%3Dsr_1_317_s

TypeError: 'NoneType' object is not subscriptable

In [3]:
df.head()

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
0,adidas,Clothing,$22.50,https://images-na.ssl-images-amazon.com/images...,$30.00,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.6
1,adidas,Basketball,$30.00,https://images-na.ssl-images-amazon.com/images...,$30.00,Men's Sport 3-Stripes Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.5
2,adidas,Soccer,$13.57 - $58.30,https://images-na.ssl-images-amazon.com/images...,$13.57 - $58.30,Men's Tastigo 19 Shorts,http://amazon.com/adidas-Tastigo-Shorts-Black-...,4.3
3,adidas,Clothing,$19.92 - $49.98,https://images-na.ssl-images-amazon.com/images...,$19.92 - $49.98,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/adidas-Designed-3-Stripes-Sh...,4.6
4,adidas,Basketball,$18.52 - $44.99,https://images-na.ssl-images-amazon.com/images...,$18.52 - $44.99,Men's Sport 3-Stripes Shorts,http://amazon.com/adidas-Sport-3-Stripes-Short...,4.5


In [4]:
# Check Images Column
df['Images'][0]

'https://images-na.ssl-images-amazon.com/images/I/71d3X3XsBLL._AC_UY879_.jpg'

In [5]:
# Check Images Column
df['Images'][1]

'https://images-na.ssl-images-amazon.com/images/I/51l2dZYdrML._AC_UX679_.jpg'

In [6]:
df.shape

(390, 8)

In [7]:
df['Brand'].describe()

count        389
unique        27
top       adidas
freq         287
Name: Brand, dtype: object

In [8]:
# Select the Adidas brand only
df=df[df['Brand'].isin(['adidas', 'adidas Originals', 'adidas outdoor'])]

In [9]:
df.shape

(307, 8)

In [10]:
df.groupby(['Brand']).count()

,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
Brand,,,,,,,
adidas,280,277,287,277,287,287,247
adidas Originals,20,20,20,20,20,20,20


In [11]:
df=df[~df['Discounted Price'].isnull()]
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
0,adidas,Clothing,$22.50,https://images-na.ssl-images-amazon.com/images...,$30.00,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.6
1,adidas,Basketball,$30.00,https://images-na.ssl-images-amazon.com/images...,$30.00,Men's Sport 3-Stripes Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.5
2,adidas,Soccer,$13.57 - $58.30,https://images-na.ssl-images-amazon.com/images...,$13.57 - $58.30,Men's Tastigo 19 Shorts,http://amazon.com/adidas-Tastigo-Shorts-Black-...,4.3
3,adidas,Clothing,$19.92 - $49.98,https://images-na.ssl-images-amazon.com/images...,$19.92 - $49.98,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/adidas-Designed-3-Stripes-Sh...,4.6
4,adidas,Basketball,$18.52 - $44.99,https://images-na.ssl-images-amazon.com/images...,$18.52 - $44.99,Men's Sport 3-Stripes Shorts,http://amazon.com/adidas-Sport-3-Stripes-Short...,4.5
...,...,...,...,...,...,...,...,...
380,adidas,Basketball,$31.06 - $55.53,https://images-na.ssl-images-amazon.com/images...,$31.06 - $55.53,Men's Dame Short,http://amazon.com/adidas-Mens-Short-Black-Smal...,4.3
381,adidas,Clothing,$31.25 - $35.00,https://images-na.ssl-images-amazon.com/images...,$31.25 - $35.00,Men's City Long Shorts,http://amazon.com/adidas-City-Long-Short-Large...,4.2
382,adidas,Clothing,$25.40 - $80.00,https://images-na.ssl-images-amazon.com/images...,$25.40 - $80.00,Men's 3-Stripes Collection Shorts,http://amazon.com/adidas-3-Stripes-Collection-...,3.5
383,adidas,Clothing,$40.00,https://images-na.ssl-images-amazon.com/images...,$40.00,Men's Training Camo Hype Woven Short,http://amazon.com/adidas-Training-Woven-Myster...,NaN


In [12]:
df=df[~df['Original Price'].isnull()]
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
0,adidas,Clothing,$22.50,https://images-na.ssl-images-amazon.com/images...,$30.00,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.6
1,adidas,Basketball,$30.00,https://images-na.ssl-images-amazon.com/images...,$30.00,Men's Sport 3-Stripes Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.5
2,adidas,Soccer,$13.57 - $58.30,https://images-na.ssl-images-amazon.com/images...,$13.57 - $58.30,Men's Tastigo 19 Shorts,http://amazon.com/adidas-Tastigo-Shorts-Black-...,4.3
3,adidas,Clothing,$19.92 - $49.98,https://images-na.ssl-images-amazon.com/images...,$19.92 - $49.98,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/adidas-Designed-3-Stripes-Sh...,4.6
4,adidas,Basketball,$18.52 - $44.99,https://images-na.ssl-images-amazon.com/images...,$18.52 - $44.99,Men's Sport 3-Stripes Shorts,http://amazon.com/adidas-Sport-3-Stripes-Short...,4.5
...,...,...,...,...,...,...,...,...
380,adidas,Basketball,$31.06 - $55.53,https://images-na.ssl-images-amazon.com/images...,$31.06 - $55.53,Men's Dame Short,http://amazon.com/adidas-Mens-Short-Black-Smal...,4.3
381,adidas,Clothing,$31.25 - $35.00,https://images-na.ssl-images-amazon.com/images...,$31.25 - $35.00,Men's City Long Shorts,http://amazon.com/adidas-City-Long-Short-Large...,4.2
382,adidas,Clothing,$25.40 - $80.00,https://images-na.ssl-images-amazon.com/images...,$25.40 - $80.00,Men's 3-Stripes Collection Shorts,http://amazon.com/adidas-3-Stripes-Collection-...,3.5
383,adidas,Clothing,$40.00,https://images-na.ssl-images-amazon.com/images...,$40.00,Men's Training Camo Hype Woven Short,http://amazon.com/adidas-Training-Woven-Myster...,NaN


In [13]:
df['Discounted Price'] = df['Discounted Price'].str.replace('$', '')
df['Discounted Price']

0              22.50
1              30.00
2      13.57 - 58.30
3      19.92 - 49.98
4      18.52 - 44.99
           ...      
380    31.06 - 55.53
381    31.25 - 35.00
382    25.40 - 80.00
383            40.00
384           103.55
Name: Discounted Price, Length: 297, dtype: object

In [14]:
df['Original Price'] = df['Original Price'].str.replace('$', '')
df['Original Price']

0              30.00
1              30.00
2      13.57 - 58.30
3      19.92 - 49.98
4      18.52 - 44.99
           ...      
380    31.06 - 55.53
381    31.25 - 35.00
382    25.40 - 80.00
383            40.00
384           103.55
Name: Original Price, Length: 297, dtype: object

In [15]:
df.dtypes

Brand               object
Category            object
Discounted Price    object
Images              object
Original Price      object
Product Name        object
Product URL         object
Ratings             object
dtype: object

In [16]:
df['Discounted Price'] = df['Discounted Price'].str.extract(r'(.....)')
df['Discounted Price'] 

0      22.50
1      30.00
2      13.57
3      19.92
4      18.52
       ...  
380    31.06
381    31.25
382    25.40
383    40.00
384    103.5
Name: Discounted Price, Length: 297, dtype: object

In [17]:
df['Original Price'] = df['Original Price'].str.extract(r'(.....)')
df['Original Price'] 

0       30.0
1      30.00
2      13.57
3      19.92
4      18.52
       ...  
380    31.06
381    31.25
382    25.40
383    40.00
384    103.5
Name: Original Price, Length: 297, dtype: object

In [18]:
df['Discounted Price'] = df['Discounted Price'].str.replace(',', '')
df['Discounted Price']

0      22.50
1      30.00
2      13.57
3      19.92
4      18.52
       ...  
380    31.06
381    31.25
382    25.40
383    40.00
384    103.5
Name: Discounted Price, Length: 297, dtype: object

In [19]:
df['Original Price'] = df['Original Price'].str.replace(',', '')
df['Original Price']

0       30.0
1      30.00
2      13.57
3      19.92
4      18.52
       ...  
380    31.06
381    31.25
382    25.40
383    40.00
384    103.5
Name: Original Price, Length: 297, dtype: object

In [20]:
df.to_csv('Amazon_adidas_shortpants3.csv', index = True)

In [21]:
df = pd.read_csv('Amazon_adidas_shortpants3.csv', index_col=0)

In [22]:
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
0,adidas,Clothing,22.50,https://images-na.ssl-images-amazon.com/images...,30.00,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.6
1,adidas,Basketball,30.00,https://images-na.ssl-images-amazon.com/images...,30.00,Men's Sport 3-Stripes Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.5
2,adidas,Soccer,13.57,https://images-na.ssl-images-amazon.com/images...,13.57,Men's Tastigo 19 Shorts,http://amazon.com/adidas-Tastigo-Shorts-Black-...,4.3
3,adidas,Clothing,19.92,https://images-na.ssl-images-amazon.com/images...,19.92,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/adidas-Designed-3-Stripes-Sh...,4.6
4,adidas,Basketball,18.52,https://images-na.ssl-images-amazon.com/images...,18.52,Men's Sport 3-Stripes Shorts,http://amazon.com/adidas-Sport-3-Stripes-Short...,4.5
...,...,...,...,...,...,...,...,...
380,adidas,Basketball,31.06,https://images-na.ssl-images-amazon.com/images...,31.06,Men's Dame Short,http://amazon.com/adidas-Mens-Short-Black-Smal...,4.3
381,adidas,Clothing,31.25,https://images-na.ssl-images-amazon.com/images...,31.25,Men's City Long Shorts,http://amazon.com/adidas-City-Long-Short-Large...,4.2
382,adidas,Clothing,25.40,https://images-na.ssl-images-amazon.com/images...,25.40,Men's 3-Stripes Collection Shorts,http://amazon.com/adidas-3-Stripes-Collection-...,3.5
383,adidas,Clothing,40.00,https://images-na.ssl-images-amazon.com/images...,40.00,Men's Training Camo Hype Woven Short,http://amazon.com/adidas-Training-Woven-Myster...,NaN


In [23]:
df['Discounted Price'] = df['Discounted Price'].apply(lambda x: float(x))*7.78 
df['Original Price'] = df['Original Price'].apply(lambda x: float(x))*7.78 
df = df.round(2)
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
0,adidas,Clothing,175.05,https://images-na.ssl-images-amazon.com/images...,233.40,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.6
1,adidas,Basketball,233.40,https://images-na.ssl-images-amazon.com/images...,233.40,Men's Sport 3-Stripes Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.5
2,adidas,Soccer,105.57,https://images-na.ssl-images-amazon.com/images...,105.57,Men's Tastigo 19 Shorts,http://amazon.com/adidas-Tastigo-Shorts-Black-...,4.3
3,adidas,Clothing,154.98,https://images-na.ssl-images-amazon.com/images...,154.98,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/adidas-Designed-3-Stripes-Sh...,4.6
4,adidas,Basketball,144.09,https://images-na.ssl-images-amazon.com/images...,144.09,Men's Sport 3-Stripes Shorts,http://amazon.com/adidas-Sport-3-Stripes-Short...,4.5
...,...,...,...,...,...,...,...,...
380,adidas,Basketball,241.65,https://images-na.ssl-images-amazon.com/images...,241.65,Men's Dame Short,http://amazon.com/adidas-Mens-Short-Black-Smal...,4.3
381,adidas,Clothing,243.12,https://images-na.ssl-images-amazon.com/images...,243.12,Men's City Long Shorts,http://amazon.com/adidas-City-Long-Short-Large...,4.2
382,adidas,Clothing,197.61,https://images-na.ssl-images-amazon.com/images...,197.61,Men's 3-Stripes Collection Shorts,http://amazon.com/adidas-3-Stripes-Collection-...,3.5
383,adidas,Clothing,311.20,https://images-na.ssl-images-amazon.com/images...,311.20,Men's Training Camo Hype Woven Short,http://amazon.com/adidas-Training-Woven-Myster...,NaN


In [24]:
df.to_csv('Amazon_adidas_shortpants3A.csv', index = True)

In [25]:
df = pd.read_csv('Amazon_adidas_shortpants3A.csv', index_col=0)

In [27]:
df = df.drop(index=[117, 141, 205, 305, 329, 354],  axis=0)
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
0,adidas,Clothing,175.05,https://images-na.ssl-images-amazon.com/images...,233.40,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.6
1,adidas,Basketball,233.40,https://images-na.ssl-images-amazon.com/images...,233.40,Men's Sport 3-Stripes Shorts,http://amazon.com/gp/slredirect/picassoRedirec...,4.5
2,adidas,Soccer,105.57,https://images-na.ssl-images-amazon.com/images...,105.57,Men's Tastigo 19 Shorts,http://amazon.com/adidas-Tastigo-Shorts-Black-...,4.3
3,adidas,Clothing,154.98,https://images-na.ssl-images-amazon.com/images...,154.98,Men's Designed 2 Move 3-Stripes Cool Shorts,http://amazon.com/adidas-Designed-3-Stripes-Sh...,4.6
4,adidas,Basketball,144.09,https://images-na.ssl-images-amazon.com/images...,144.09,Men's Sport 3-Stripes Shorts,http://amazon.com/adidas-Sport-3-Stripes-Short...,4.5
...,...,...,...,...,...,...,...,...
380,adidas,Basketball,241.65,https://images-na.ssl-images-amazon.com/images...,241.65,Men's Dame Short,http://amazon.com/adidas-Mens-Short-Black-Smal...,4.3
381,adidas,Clothing,243.12,https://images-na.ssl-images-amazon.com/images...,243.12,Men's City Long Shorts,http://amazon.com/adidas-City-Long-Short-Large...,4.2
382,adidas,Clothing,197.61,https://images-na.ssl-images-amazon.com/images...,197.61,Men's 3-Stripes Collection Shorts,http://amazon.com/adidas-3-Stripes-Collection-...,3.5
383,adidas,Clothing,311.20,https://images-na.ssl-images-amazon.com/images...,311.20,Men's Training Camo Hype Woven Short,http://amazon.com/adidas-Training-Woven-Myster...,NaN


In [28]:
df.to_csv('Amazon_adidas_shortpants3A_final.csv', index = True)